In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ======================== CELL 1: DEPENDENCIES ==========================
import subprocess
import sys

print('🔧 Installing dependencies...')
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"], capture_output=True, check=False)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyarrow==15.0.2", "keybert", "rank-bm25", "faiss-cpu", "sacremoses"], check=True)
print('✅ Dependencies installed. RESTART kernel and run Cell 2.')


In [10]:
# ===================== CELL 2: IMPORTS & SETUP =====================
import warnings
warnings.filterwarnings("ignore")

import re
import json
import time
import pickle
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Tuple

import numpy as np
import torch
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize, sent_tokenize
from rank_bm25 import BM25Okapi

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# ============================================================================
# CONFIGURATION FOR NUTRITION & LIFESTYLE DOMAIN
# ============================================================================

@dataclass
class DomainConfig:
    name: str = "ophthalmology"
    dataset_name: str = "BaekSeungJu/Ophthalmology-EQA-v3"
    dataset_split: str = "train"
    chunk_window: int = 3
    chunk_stride: int = 1
    embed_model: str = "sentence-transformers/all-MiniLM-L6-v2"

config = DomainConfig()

print(f"📋 Building index for: {config.name}")
print(f"📦 Dataset: {config.dataset_name}")


🔧 Using device: cpu
📋 Building index for: ophthalmology
📦 Dataset: BaekSeungJu/Ophthalmology-EQA-v3


In [11]:
# ===================== CELL 3: DATASET LOADING =====================
def extract_qa_pairs(dataset) -> list:
    qa_data = []
    for idx, row in enumerate(dataset):
        try:
            if 'question' in row and 'answer' in row:
                qa_data.append({
                    "question": str(row['question']).strip(),
                    "answer": str(row['answer']).strip(),
                    "source_id": idx
                })
            elif 'instruction' in row and 'output' in row:
                qa_data.append({
                    "question": str(row['instruction']).strip(),
                    "answer": str(row['output']).strip(),
                    "source_id": idx
                })
        except Exception as e:
            if idx < 3:
                print(f"⚠️ Row {idx} skipped: {e}")
    return qa_data


# Load dataset
print(f"📥 Loading {config.dataset_name}...")
dataset = load_dataset(config.dataset_name, split=config.dataset_split)
print(f"✅ Loaded {len(dataset)} rows")

# Print sample row to verify structure
print(f"\n📋 Sample row structure:")
print(f"Keys: {list(dataset[0].keys())}")

# Extract Q&A pairs
print(f"\n🔍 Extracting nutrition & lifestyle Q&A pairs...")
qa_data = extract_qa_pairs(dataset)
print(f"✅ Extracted {len(qa_data)} Q&A pairs")

if len(qa_data) < 100:
    print(f"⚠️ WARNING: Only {len(qa_data)} pairs extracted!")
else:
    print(f"\n📝 Sample nutrition Q&A:")
    if qa_data:
        print(f"Q: {qa_data[0]['question'][:150]}...")
        print(f"A: {qa_data[0]['answer'][:150]}...")


📥 Loading BaekSeungJu/Ophthalmology-EQA-v3...


README.md:   0%|          | 0.00/317 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49300 [00:00<?, ? examples/s]

✅ Loaded 49300 rows

📋 Sample row structure:
Keys: ['question', 'answer']

🔍 Extracting nutrition & lifestyle Q&A pairs...
✅ Extracted 49300 Q&A pairs

📝 Sample nutrition Q&A:
Q: How can infections spread from the ethmoid sinus to the orbit?...
A: Infections can spread from the ethmoid sinus to the orbit through the emissary channels in the medial wall of the orbit....


In [12]:
def create_chunks(data: List[Dict], window: int = 3, stride: int = 1, min_chars: int = 10) -> List[Dict]:
    """Create sentence-level chunks from answers."""
    chunks = []
    for item in data:
        text = item.get("answer", "")
        if not text or len(text) < min_chars:  # Lower to 10 chars
            continue

        sentences = sent_tokenize(text)
        if not sentences:
            continue

        if len(sentences) <= window:
            chunks.append({
                "chunk": " ".join(sentences),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
            continue

        for i in range(0, max(1, len(sentences) - window + 1), stride):
            chunks.append({
                "chunk": " ".join(sentences[i:i + window]),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
    return chunks

print(f"🔪 Creating chunks (window={config.chunk_window}, stride={config.chunk_stride}, min_chars=10)...")
chunks = create_chunks(qa_data, window=config.chunk_window, stride=config.chunk_stride, min_chars=10)
print(f"✅ Created {len(chunks)} chunks")

if len(chunks) == 0:
    print("[❌ ERROR] No chunks created! Review the extracted sample answers, min_chars, or dataset format.")
else:
    print("\n📝 Sample chunk:")
    for i, chunk in enumerate(chunks[:3]):
        print(f"{i+1}. {chunk['chunk'][:120]}...")


🔪 Creating chunks (window=3, stride=1, min_chars=10)...
✅ Created 57979 chunks

📝 Sample chunk:
1. Infections can spread from the ethmoid sinus to the orbit through the emissary channels in the medial wall of the orbit....
2. Fractures of the orbital floor involving the infraorbital groove, which contains the infraorbital nerve, should be suspe...
3. The optic canal transmits the optic nerve and the ophthalmic artery....


In [13]:
# ===================== CELL 5: BUILD FAISS INDEX =====================

# Load embedder
print(f"📦 Loading embedder: {config.embed_model}")
embedder = SentenceTransformer(config.embed_model, device=device)
print(f"✅ Embedder loaded")

# Extract chunk texts
id2doc = [chunk["chunk"] for chunk in chunks]
print(f"📊 Encoding {len(id2doc)} chunks...")

# Create embeddings
embeddings = embedder.encode(
    id2doc,
    normalize_embeddings=True,
    show_progress_bar=True,
    batch_size=64,
    convert_to_numpy=True
).astype('float32')

print(f"✅ Embeddings shape: {embeddings.shape}")

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print(f"✅ FAISS index built: {index.ntotal} vectors, dimension {dim}")


📦 Loading embedder: sentence-transformers/all-MiniLM-L6-v2
✅ Embedder loaded
📊 Encoding 57979 chunks...


Batches:   0%|          | 0/906 [00:00<?, ?it/s]

✅ Embeddings shape: (57979, 384)
✅ FAISS index built: 57979 vectors, dimension 384


In [14]:
# ===================== CELL 6: BUILD BM25 INDEX =====================

# Tokenize for BM25
print(f"🔨 Building BM25 index...")
bm25_corpus = [word_tokenize(doc.lower()) for doc in id2doc]
bm25 = BM25Okapi(bm25_corpus)
print(f"✅ BM25 index built")


🔨 Building BM25 index...
✅ BM25 index built


In [15]:
# ===================== CELL 7: SAVE INDEXES =====================

# File names
index_file = f"{config.name}_faiss.index"
id2doc_file = f"{config.name}_id2doc.pkl"
metadata_file = f"{config.name}_metadata.json"

# Save FAISS index
print(f"💾 Saving FAISS index to {index_file}...")
faiss.write_index(index, index_file)

# Save id2doc mapping
print(f"💾 Saving id2doc to {id2doc_file}...")
with open(id2doc_file, "wb") as f:
    pickle.dump(id2doc, f)

# Save metadata
metadata = {
    "created_at": time.time(),
    "domain": config.name,
    "dataset": config.dataset_name,
    "n_vectors": int(index.ntotal),
    "embedding_dim": dim,
    "chunk_window": config.chunk_window,
    "chunk_stride": config.chunk_stride,
    "embed_model": config.embed_model
}

print(f"💾 Saving metadata to {metadata_file}...")
with open(metadata_file, "w") as f:
    json.dump(metadata, f, indent=2)

print("\n" + "="*80)
print("✅ INDEX BUILDING COMPLETE!")
print("="*80)
print(f"📁 Files created:")
print(f"  - {index_file}")
print(f"  - {id2doc_file}")
print(f"  - {metadata_file}")
print(f"\n📊 Statistics:")
print(f"  - Total vectors: {index.ntotal}")
print(f"  - Embedding dimension: {dim}")
print(f"  - Source Q&A pairs: {len(qa_data)}")
print(f"  - Chunks created: {len(chunks)}")
print("="*80)


💾 Saving FAISS index to ophthalmology_faiss.index...
💾 Saving id2doc to ophthalmology_id2doc.pkl...
💾 Saving metadata to ophthalmology_metadata.json...

✅ INDEX BUILDING COMPLETE!
📁 Files created:
  - ophthalmology_faiss.index
  - ophthalmology_id2doc.pkl
  - ophthalmology_metadata.json

📊 Statistics:
  - Total vectors: 57979
  - Embedding dimension: 384
  - Source Q&A pairs: 49300
  - Chunks created: 57979


In [16]:
# ===================== CELL 8: VERIFY INDEX (OPTIONAL) =====================

def test_retrieval(query: str, top_k: int = 5):
    """Test the built index with a sample query."""
    print(f"\n🔍 Testing query: '{query}'")
    
    # Embed query
    query_emb = embedder.encode([query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
    
    # Search FAISS
    D, I = index.search(query_emb, top_k)
    
    print(f"\n📋 Top {top_k} results:")
    for i, (idx, score) in enumerate(zip(I[0], D[0]), 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"   {id2doc[idx][:200]}...")

# Test queries
test_queries = [
    "What foods are good for heart health?",
    "How much protein should I eat daily?",
    "What are the benefits of a Mediterranean diet?"
]

print("\n" + "="*80)
print("🧪 TESTING NUTRITION INDEX")
print("="*80)

for query in test_queries:
    test_retrieval(query, top_k=3)



🧪 TESTING NUTRITION INDEX

🔍 Testing query: 'What foods are good for heart health?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.4794
   For individuals at risk of advanced ARMD, it is recommended to consume green leafy vegetables and fish or fish oils high in omega-3 fatty acids, provided it is approved by their primary care physician...

2. Score: 0.4438
   Foods rich in omega-3s include certain fish, crustaceans, flaxseed oil, dark leafy greens, and walnuts. The DREAM Study compared omega-3 supplementation with a placebo of olive oil and found no statis...

3. Score: 0.4187
   Omega-3 fatty acids have been shown to increase tear production and volume by inhibiting proinflammatory eicosanoids and cytokines. Foods rich in omega-3s include certain fish, crustaceans, flaxseed o...

🔍 Testing query: 'How much protein should I eat daily?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.3875
   The dosage for ε-aminocaproic acid is typically 50–100 mg/kg every 4 hours, up to a maximum of 30 g daily. The treatment should be continued for 5 to 6 days to achieve maximum clinical effectiveness....

2. Score: 0.3493
   Treatment includes reducing dietary protein and substituting with artificially flavored solutions of essential amino acids without arginine, while monitoring serum ammonia levels. Supplemental vitamin...

3. Score: 0.3394
   Proteins make up 33% of the lens's weight, which is significantly higher than in other body tissues. In some animal species, proteins can constitute more than 50% of the lens weight....

🔍 Testing query: 'What are the benefits of a Mediterranean diet?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.4198
   The role of nutrition, particularly the potential benefit of antioxidant supplementation, in cataract prevention has been a topic of interest and debate, as mentioned in the text....

2. Score: 0.4044
   In developing regions, animal sources of vitamin A are often scarce, and environmental factors significantly impact dietary quality. Social factors, such as inadequate water supply, can lead to chroni...

3. Score: 0.3831
   Dietary therapy can be effective in managing genetic disorders affecting the eye by addressing the specific metabolic deficiencies. For example, a galactose-free diet can reverse some clinical signs o...
